**###IMPORTAR LIBRERIAS NECESARIAS**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import csv
import base64
from io import BytesIO
from datetime import datetime, timedelta
import logging
import numpy as np
import pandas as pd
import requests
import json
from typing import Dict, Any, List
import random
import uuid

###VARIABLES

In [2]:
API_URL = "https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/estacions-contaminacio-atmosferiques-estaciones-contaminacion-atmosfericas/records?limit=20"  # Reemplaza con la URL real de tu API
OUTPUT_DIR = "datos_calidad_aire"  # Directorio para guardar los archivos CSV de contaminación y atmosfericas
API_URL2 = "https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/estacions-atmosferiques-estaciones-atmosfericas/records?limit=20"


In [3]:
camera_fiware_mapping = {
    'AVFRANCIA': ["10302", "10304", "10305"], 
    'BULEVARDSUD': ["11502", "11504", "11505"], 
    'MOLISOL': ["11402", "11403", "11404"], 
    'PISTASILLA': ["2602", "2604", "2605"], 
    'VIVERS': ["3103", "3105", "3106"], 
    'CENTRE': ["10510", "10511", "10512"], 
    'DR_LLUCH': ["14203", "14204", "14206"], 
    'OLIVERETA': ["6004", "6011", "6012"], 
    'PATRAIX': ["4302", "4304", "4305"],
    }

In [4]:
# Configurar logging para mostrar información detallada
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)


In [5]:

def aplanar_json(item: Dict[str, Any]) -> Dict[str, Any]:
    """
    Aplana un objeto JSON anidado en un diccionario plano.
    """
    resultado = {}
    
    # Procesar los campos principales
    for key, value in item.items():
        if key != "geo_shape" and key != "geo_point_2d":
            resultado[key] = value
    
    # Procesar geo_shape
    if "geo_shape" in item and "geometry" in item["geo_shape"]:
        if "coordinates" in item["geo_shape"]["geometry"]:
            resultado["longitude"] = item["geo_shape"]["geometry"]["coordinates"][0]
            resultado["latitude"] = item["geo_shape"]["geometry"]["coordinates"][1]
        
        if "type" in item["geo_shape"]["geometry"]:
            resultado["geometry_type"] = item["geo_shape"]["geometry"]["type"]
    
    # Procesar geo_point_2d si existe
    if "geo_point_2d" in item:
        if "lon" in item["geo_point_2d"]:
            resultado["geo_point_lon"] = item["geo_point_2d"]["lon"]
        if "lat" in item["geo_point_2d"]:
            resultado["geo_point_lat"] = item["geo_point_2d"]["lat"]
    
    # Añadir timestamp de captura
    resultado["timestamp_captura"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    
    return resultado

In [6]:
def guardar_csv(item: Dict[str, Any], es_nuevo_archivo: bool) -> str:
    """
    Guarda un item en un archivo CSV usando fiwareId como nombre.
    Retorna la ruta del archivo.
    """
    # Asegurarse de que exista el directorio de salida
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Obtener fiwareId como nombre de archivo
    fiware_id = item.get("fiwareid", f"desconocido_{item.get('objectid', 'sin_id')}")
    archivo_csv = os.path.join(OUTPUT_DIR, f"{fiware_id}.csv")
    
    # Determinar si es un archivo nuevo o existente
    modo = "w" if es_nuevo_archivo else "a"
    
    # Lista de todas las claves disponibles en los datos aplanados
    headers = list(item.keys())
    
    with open(archivo_csv, modo, newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        
        # Escribir encabezados solo si es un archivo nuevo
        if es_nuevo_archivo:
            writer.writerow(headers)
        
        # Escribir los valores
        row_values = [str(item.get(key, "")) for key in headers]
        writer.writerow(row_values)
    
    return archivo_csv


In [7]:
def obtener_datos_api(url: str) -> List[Dict[str, Any]]:
    """
    Obtiene los datos de la API especificada por URL y los devuelve como una lista de diccionarios.
    """
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        if "results" in data and isinstance(data["results"], list):
            return data["results"]
        else:
            logger.error("Estructura de datos inesperada: %s", data)
            return []
            
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al obtener datos de la API: {e}")
        return []
    except json.JSONDecodeError as e:
        logger.error(f"Error al decodificar JSON: {e}")
        return [] 


In [8]:
def procesar_datos(url: str):
    """
    Procesa los datos de la API proporcionada por URL y los guarda en archivos CSV.
    """
    logger.info("Iniciando captura de datos...")
    
    items = obtener_datos_api(url)
    
    if not items:
        logger.warning("No se obtuvieron datos en esta ejecución")
        return
    
    logger.info(f"Obtenidos {len(items)} registros")
    
    archivos_existentes = set(
        os.path.splitext(f)[0] 
        for f in os.listdir(OUTPUT_DIR) 
        if os.path.isfile(os.path.join(OUTPUT_DIR, f)) and f.endswith('.csv')
    ) if os.path.exists(OUTPUT_DIR) else set()
    
    for item in items:
        item_plano = aplanar_json(item)
        
        fiware_id = item.get("fiwareid", f"desconocido_{item.get('objectid', 'sin_id')}")
        es_nuevo_archivo = fiware_id not in archivos_existentes
        
        ruta_archivo = guardar_csv(item_plano, es_nuevo_archivo)
        
        logger.info(f"Datos guardados en {ruta_archivo} (nuevo: {es_nuevo_archivo})")


In [9]:
def capture_video_element(camera_id, camera_folder):
    """
    Captura un frame de video de una cámara con estrategia de reintentos y rotación de user-agents.
    Si después de 5 segundos no hay contenido, reinicia el navegador con un nuevo perfil.
    """
    logger.info(f"Iniciando captura de video para cámara {camera_id}")
    
    # Timestamp consistente para toda la función
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    screenshot_filename = os.path.join(camera_folder, f"captura_{camera_id}_{timestamp}.png")
    
    # Lista de user-agents para rotar y simular diferentes navegadores/dispositivos
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/96.0.1054.43 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 15_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/96.0.4664.53 Mobile/15E148 Safari/604.1',
        'Mozilla/5.0 (iPad; CPU OS 15_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1'
    ]
    
    # Función para obtener un navegador configurado con options personalizadas
    def get_browser():
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--disable-extensions')
        options.add_argument('--disable-logging')
        options.add_argument('--disable-infobars')
        options.add_argument('--disable-notifications')
        
        # Rotar user agent
        user_agent = random.choice(user_agents)
        options.add_argument(f'--user-agent={user_agent}')
        
        # Añadir cabeceras aleatorias para simular un navegador diferente
        options.add_argument(f'--accept-language={"en-US,en;q=0.9" if random.random() > 0.5 else "es-ES,es;q=0.9"}')
        
        # Añadir cabeceras personalizadas adicionales
        options.add_experimental_option("prefs", {
            "profile.default_content_setting_values.cookies": random.choice([1, 2]),  # 1=permitir, 2=bloquear
            "profile.default_content_setting_values.images": 1,  # Cargar imágenes
            "profile.default_content_setting_values.javascript": 1  # Habilitar JavaScript
        })
        
        # Generar ID de sesión aleatoria para evitar caché
        session_id = str(uuid.uuid4())
        options.add_argument(f'--profile-directory=Profile{random.randint(1, 999)}')
        
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Script JavaScript para detectar contenido en el video
    canvas_script = """
    var video = arguments[0];
    var canvas = document.createElement('canvas');
    
    // Usar el tamaño real del video
    var videoWidth = video.videoWidth || video.clientWidth || 640;
    var videoHeight = video.videoHeight || video.clientHeight || 480;
    
    canvas.width = videoWidth;
    canvas.height = videoHeight;
    
    var ctx = canvas.getContext('2d');
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    
    // Verificación optimizada de contenido
    var imageData = ctx.getImageData(0, 0, canvas.width, canvas.height);
    var data = imageData.data;
    var hasContent = false;
    
    // Verificación en rejilla: comprobamos puntos distribuidos por toda la imagen
    var stride = Math.floor(data.length / 100); // Aproximadamente 25 puntos
    for (var i = 0; i < data.length; i += stride) {
        if ((data[i] > 10 || data[i+1] > 10 || data[i+2] > 10) && data[i+3] > 0) {
            hasContent = true;
            break;
        }
    }
    
    return {
        dataUrl: canvas.toDataURL('image/png'),
        hasContent: hasContent
    };
    """
    
    # Intento inicial
    max_attempts = 2  # Intentos totales (inicial + 1 reintento)
    current_attempt = 0
    
    while current_attempt < max_attempts:
        current_attempt += 1
        logger.info(f"Intento #{current_attempt} para cámara {camera_id}")
        
        driver = get_browser()
        
        try:
            # URL de la cámara
            url = f"https://camaras.valencia.es/camaras/visualizador.html?id={camera_id}"
            logger.info(f"Accediendo a {url}")
            driver.get(url)
            
            # Esperar a que el elemento video esté presente y visible
            logger.info("Esperando a que cargue el stream de video...")
            wait = WebDriverWait(driver, 10)
            
            # Intentar encontrar el elemento video
            try:
                video_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "video#test-video")))
                logger.info("Elemento de video encontrado con selector: video#test-video")
            except Exception:
                # Si falla, intentar con selectores alternativos
                video_element = None
                for selector in ["video.test-video", "video[controls]", "video"]:
                    try:
                        video_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
                        logger.info(f"Elemento de video encontrado con selector: {selector}")
                        break
                    except:
                        continue
            
            if not video_element:
                logger.error("No se pudo encontrar el elemento de video")
                debug_path = os.path.join(camera_folder, f"debug_page_{camera_id}_{timestamp}_attempt{current_attempt}.png")
                driver.save_screenshot(debug_path)
                logger.info(f"Captura de depuración guardada: {debug_path}")
                driver.quit()
                
                if current_attempt < max_attempts:
                    logger.info("Reintentando con un nuevo navegador...")
                    continue
                else:
                    return None, timestamp
            
            # Lógica para el primer intento - esperar máximo 5 segundos
            if current_attempt == 1:
                first_attempt_max_wait = 5  # Segundos
                start_time = time.time()
                
                # Espera inicial reducida
                time.sleep(0.5)
                
                # Bucle para el primer intento
                while (time.time() - start_time) < first_attempt_max_wait:
                    try:
                        result = driver.execute_script(canvas_script, video_element)
                        
                        if result.get('hasContent', False):
                            logger.info("¡Frame con contenido detectado!")
                            
                            # Procesar y guardar la imagen
                            img_base64 = result['dataUrl']
                            img_data = img_base64.split(',')[1]
                            img_bytes = base64.b64decode(img_data)
                            img = Image.open(BytesIO(img_bytes))
                            img.save(screenshot_filename)
                            logger.info(f"Frame de video capturado y guardado como {screenshot_filename}")
                            driver.quit()
                            return screenshot_filename, timestamp
                        else:
                            logger.info("Frame vacío o negro detectado, esperando...")
                            time.sleep(0.5)
                    
                    except Exception as e:
                        logger.warning(f"Error al intentar capturar frame: {str(e)}")
                        time.sleep(0.5)
                
                logger.info(f"Tiempo de espera inicial ({first_attempt_max_wait}s) agotado sin obtener frame con contenido")
                logger.info("Cerrando navegador y reintentando con nuevo perfil...")
                driver.quit()
                
                # Continuar con el siguiente intento (solo si estamos en el primero)
                if current_attempt < max_attempts:
                    continue
            
            # Lógica para el segundo intento - esperar máximo 10 segundos
            else:
                second_attempt_max_wait = 10  # Segundos
                start_time = time.time()
                
                # Bucle para el segundo intento (más tiempo)
                while (time.time() - start_time) < second_attempt_max_wait:
                    try:
                        result = driver.execute_script(canvas_script, video_element)
                        
                        if result.get('hasContent', False):
                            logger.info("¡Frame con contenido detectado en segundo intento!")
                            
                            # Procesar y guardar la imagen
                            img_base64 = result['dataUrl']
                            img_data = img_base64.split(',')[1]
                            img_bytes = base64.b64decode(img_data)
                            img = Image.open(BytesIO(img_bytes))
                            img.save(screenshot_filename)
                            logger.info(f"Frame de video capturado y guardado como {screenshot_filename}")
                            driver.quit()
                            return screenshot_filename, timestamp
                        else:
                            logger.info("Frame vacío o negro detectado en segundo intento, esperando...")
                            time.sleep(1)  # Intervalo más largo en segundo intento
                    
                    except Exception as e:
                        logger.warning(f"Error al intentar capturar frame en segundo intento: {str(e)}")
                        time.sleep(0.5)
                
                logger.warning(f"Tiempo de espera extendido ({second_attempt_max_wait}s) agotado sin obtener frame con contenido")
                
                # Último recurso: captura de pantalla directa
                try:
                    driver.save_screenshot(screenshot_filename)
                    logger.info(f"Se usó captura de pantalla alternativa: {screenshot_filename}")
                    driver.quit()
                    return screenshot_filename, timestamp
                except Exception as e:
                    logger.error(f"Error en captura alternativa: {str(e)}")
                    driver.quit()
                    return None, timestamp
        
        except Exception as e:
            logger.error(f"Error en el proceso de captura (intento #{current_attempt}): {str(e)}")
            try:
                driver.quit()
            except:
                pass
            
            if current_attempt < max_attempts:
                logger.info("Reintentando con un nuevo navegador...")
                continue
            else:
                return None, timestamp
    
    # Si llegamos aquí, fallaron todos los intentos
    logger.error(f"Todos los intentos ({max_attempts}) fallaron para la cámara {camera_id}")
    return None, timestamp

In [10]:
def create_directories(camera_fiware_mapping, base_folder="capturas_camaras"):
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    camera_folders = {}
    for fiware_id, camera_ids in camera_fiware_mapping.items():
        for camera_id in camera_ids:
            folder = os.path.join(base_folder, f"camara_{camera_id}")
            os.makedirs(folder, exist_ok=True)
            camera_folders[camera_id] = folder
    return camera_folders

In [11]:
def capture_and_process(camera_id, fiware_id, folders):
    """
    Captura imagen de una cámara, procesa la imagen y detecta vehículos.
    Si no detecta nada en la imagen mejorada, reintenta con la original.
    """
    print(f"Capture -> {camera_id} {fiware_id}")
    
    # Captura imagen
    screenshot_path, capture_time_str = capture_video_element(camera_id, folders[camera_id])
    
    if not screenshot_path or not os.path.exists(screenshot_path):
        logger.warning(f"[{camera_id}] No se pudo capturar imagen. Path generado: {screenshot_path}")
        return None

    logger.info(f"[{camera_id}] Imagen capturada: {screenshot_path}")
    
    return True

    


In [12]:

def monitor_traffic(camera_fiware_mapping, duration_hours=8):
    """
    Monitorea el tráfico de una lista de cámaras y guarda los datos.
    Parámetros:
    - camera_fiware_mapping: dict de fiware_id -> lista de camera_id
    - duration_hours: duración total del monitoreo
    """
    END_TIME = datetime.now() + timedelta(hours=duration_hours)
    base_folder = "capturas_camaras"
    
    camera_folders = create_directories(camera_fiware_mapping, base_folder)

    logger.info(f"El monitoreo finalizará a las {END_TIME.strftime('%Y-%m-%d %H:%M:%S')}")
    
    capture_count = 0
    procesar_datos(API_URL)
    procesar_datos(API_URL2)
    while datetime.now() < END_TIME:

        logger.info(f"=== Iniciando ronda de capturas ===")
        
        for fiware_id, camera_ids in camera_fiware_mapping.items():
            for camera_id in camera_ids:
                try:
                    logger.info(f"Procesando cámara {camera_id}...")
                    capture_data = capture_and_process(camera_id, fiware_id, camera_folders)
                    capture_count += 1
                    if capture_data:
                        logger.info(f"Captura Exitosa {camera_id}")
                        
                except Exception as e:
                    logger.error(f"Error capturando datos de cámara {camera_id}: {str(e)}")
        
        procesar_datos(API_URL)
        procesar_datos(API_URL2)        

    logger.info(f"Monitoreo finalizado. Total Capturas: " + capture_count)

In [13]:
# Iniciar el monitoreo
if __name__ == "__main__":
    logger.info("=== Iniciando programa de monitoreo de tráfico ===")
    # Parámetros: IDs de cámaras, duración (horas)
    monitor_traffic(camera_fiware_mapping, duration_hours=1)

2025-05-13 08:39:33,577 - === Iniciando programa de monitoreo de tráfico ===
2025-05-13 08:39:33,583 - El monitoreo finalizará a las 2025-05-13 09:39:33
2025-05-13 08:39:33,584 - Iniciando captura de datos...
2025-05-13 08:39:33,797 - Obtenidos 11 registros
2025-05-13 08:39:33,804 - Datos guardados en datos_calidad_aire\A06_VIVERS_60m.csv (nuevo: False)
2025-05-13 08:39:33,807 - Datos guardados en datos_calidad_aire\A04_PISTASILLA_60m.csv (nuevo: False)
2025-05-13 08:39:33,808 - Datos guardados en datos_calidad_aire\A09_CABANYAL_60m.csv (nuevo: False)
2025-05-13 08:39:33,811 - Datos guardados en datos_calidad_aire\A10_OLIVERETA_60m.csv (nuevo: False)
2025-05-13 08:39:33,812 - Datos guardados en datos_calidad_aire\A01_AVFRANCIA_60m.csv (nuevo: False)
2025-05-13 08:39:33,813 - Datos guardados en datos_calidad_aire\A07_VALENCIACENTRE_60m.csv (nuevo: False)
2025-05-13 08:39:33,815 - Datos guardados en datos_calidad_aire\A02_BULEVARDSUD_60m.csv (nuevo: False)
2025-05-13 08:39:33,816 - Datos

Capture -> 10302 AVFRANCIA


2025-05-13 08:39:36,424 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:39:36,515 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:39:36,586 - There is no [win64] chromedriver "136.0.7103.92" for browser google-chrome "136.0.7103" in cache
2025-05-13 08:39:36,587 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:39:36,844 - WebDriver version 136.0.7103.92 selected
2025-05-13 08:39:36,847 - Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.92/win32/chromedriver-win32.zip
2025-05-13 08:39:36,848 - About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.92/win32/chromedriver-win32.zip
2025-05-13 08:39:36,936 - Driver downloading response is 200
2025-05-13 08:39:37,207 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:39:38,083 - Driver has been saved in cache [C:\Users\foro_\.wdm\drivers\chromedriver\win64\136.0.7103.92]
2025-05-13 08:39:

Capture -> 10304 AVFRANCIA


2025-05-13 08:40:04,470 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:40:04,535 - Get LATEST chromedriver version for google-chrome
2025-05-13 08:40:04,603 - Driver [C:\Users\foro_\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe] found in cache
2025-05-13 08:40:05,686 - Accediendo a https://camaras.valencia.es/camaras/visualizador.html?id=10304
2025-05-13 08:40:06,032 - Esperando a que cargue el stream de video...
2025-05-13 08:40:06,050 - Elemento de video encontrado con selector: video#test-video
2025-05-13 08:40:06,563 - Frame vacío o negro detectado, esperando...
2025-05-13 08:40:07,091 - Frame vacío o negro detectado, esperando...
2025-05-13 08:40:07,606 - Frame vacío o negro detectado, esperando...
2025-05-13 08:40:08,130 - Frame vacío o negro detectado, esperando...
2025-05-13 08:40:08,638 - Frame vacío o negro detectado, esperando...
2025-05-13 08:40:09,167 - ¡Frame con contenido detectado!
2025-05-13 08:40:09,168 - Error al

...

Capture -> 4304 PATRAIX


2025-05-13 09:48:56,887 - Get LATEST chromedriver version for google-chrome
2025-05-13 09:48:57,100 - Get LATEST chromedriver version for google-chrome
2025-05-13 09:48:57,180 - Driver [C:\Users\foro_\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe] found in cache
2025-05-13 09:48:58,280 - Accediendo a https://camaras.valencia.es/camaras/visualizador.html?id=4304
2025-05-13 09:48:58,933 - Esperando a que cargue el stream de video...
2025-05-13 09:48:58,965 - Elemento de video encontrado con selector: video#test-video
2025-05-13 09:48:59,501 - Frame vacío o negro detectado, esperando...
2025-05-13 09:49:00,021 - ¡Frame con contenido detectado!
2025-05-13 09:49:00,022 - Error al intentar capturar frame: name 'Image' is not defined
2025-05-13 09:49:00,554 - ¡Frame con contenido detectado!
2025-05-13 09:49:00,555 - Error al intentar capturar frame: name 'Image' is not defined
2025-05-13 09:49:01,087 - ¡Frame con contenido detectado!
2025-05-13 09:49:01,088

Capture -> 4305 PATRAIX


2025-05-13 09:49:22,790 - Get LATEST chromedriver version for google-chrome
2025-05-13 09:49:22,852 - Get LATEST chromedriver version for google-chrome
2025-05-13 09:49:22,925 - Driver [C:\Users\foro_\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe] found in cache
2025-05-13 09:49:24,028 - Accediendo a https://camaras.valencia.es/camaras/visualizador.html?id=4305
2025-05-13 09:49:24,363 - Esperando a que cargue el stream de video...
2025-05-13 09:49:24,380 - Elemento de video encontrado con selector: video#test-video
2025-05-13 09:49:24,902 - Frame vacío o negro detectado, esperando...
2025-05-13 09:49:25,439 - ¡Frame con contenido detectado!
2025-05-13 09:49:25,440 - Error al intentar capturar frame: name 'Image' is not defined
2025-05-13 09:49:25,968 - ¡Frame con contenido detectado!
2025-05-13 09:49:25,969 - Error al intentar capturar frame: name 'Image' is not defined
2025-05-13 09:49:26,514 - ¡Frame con contenido detectado!
2025-05-13 09:49:26,515

TypeError: can only concatenate str (not "int") to str